# Plotting with Matplotlib and Seaborn

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import biopsykit as bp
import biopsykit.saliva as saliva

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close('all')

## Matplotlib

Importing the `pyplot` interface from Matplotlib:  
`import matplotlib.pyplot as plt`

`pyplot`: procedural interface of the object-oriented plot library of Matplotlib. The commands of `pyplot` are chosen to be similar to *MATLAB* in both: their names and their arguments.

In order to display Matplotlib plots inside the notebook, and to interact with them (zooming, etc.), you need to enable the JupyterLab Matplotlib widget:  
`%matplotlib widget`

### Introduction

A plot consists of a `figure` (e.g. a window, a Jupyter widget, etc.) with one or more `axes` (an area in which data are plotted).

Usage Guide: https://matplotlib.org/tutorials/introductory/usage.html#sphx-glr-tutorials-introductory-usage-py

*Matplotlib* supports both an *object-oriented* as well as an *imperative* syntax for plotting. The imperative syntax was deliberately designed to be very similar to the *MATLAB* syntax and works on the global `plt` module ("state machine"). The object-oriented syntax is very similar to the imperative syntax, but unlike the imperative syntax it works on `Figure` and `Axes` objects that belong to the respective plot.

<div class="alert alert-block alert-info">
    <b>Note:</b> The object-oriented syntax is usually more intuitive to use, since the plot functions have an effect on a specific object and not on the global <code>plt</code> module.
</div>

In [ ]:
x = np.arange(0, 2, 0.01)
y = np.sin(4 * np.pi * x)

#### Imperative syntax

In [ ]:
plt.figure() # Creating a new figure
plt.plot(x, y)
plt.xlabel('Time [s]')
plt.ylabel('Amplitude [V]')
plt.title('Simple plot - imperative syntax')
plt.grid(True)

#### Object-oriented syntax

In [ ]:
fig, ax = plt.subplots() # Creating a Figure object and associated Axes
ax.plot(x, y)
ax.set_xlabel("x-axis")
ax.set_ylabel("y-axis")
ax.set_title("Simple plot - object-oriented syntax")
ax.grid(True)

fig.tight_layout()

<div class="alert alert-block alert-info">
    <b>Note:</b> When the warning
    <p><code>RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (matplotlib.pyplot.figure) are retained until explicitly closed and may consume too much memory.</code></p>
    appears, it simply means that during the execution of the notebook more and more new figures were created (e.g. by always calling <code>plt.figure()</code> or <code>plt.subplots()</code>). By calling <code>plt.close('all')</code> once (which is placed e.g. at the beginning of the notebook in the cell where external libraries are also imported) one closes all Figures opened in the background.
</div>

### Simple plots

Loading data and calculating mean value per group:

In [ ]:
data = pd.read_csv('data/cortisol_sample.csv', index_col='subject')
data_mean = data.groupby('condition').mean()
data_mean

In [ ]:
#fig, ax = plt.subplots() # Creating a Figure and associated Axes
fig, ax = plt.subplots(figsize=(8, 5)) # Specifying the size of the figure (and aspect ratio)
ax.plot(data_mean.xs('Control'), 'o-') # Plotting the control group. Here: 'o-' to plot line and marker of the data.
ax.set_xlabel("Cortisol Sample") # Title of the x-axis
ax.set_ylabel("Cortisol [nmol/l]") # Title of the y-axis
ax.set_ylim([5, 8]) # y-axis limit
ax.tick_params(bottom=True, left=True) # Activating the ticks of the x and y axes

Multiple lines in one Axes:

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5)) # Specifying the size of the figure (and aspect ratio)
ax.plot(data_mean.xs('Control'), 'o-', label="Control") # Plotting the control group
ax.plot(data_mean.xs('Intervention'), 'o--', label="Intervention") # Plotting the intervention group; -- = dashed line
ax.set_xlabel("Cortisol Sample") # Title of the x-axis
ax.set_ylabel("Cortisol [nmol/l]") # Title of the y-axis
ax.set_ylim([3, 8]) # y-axis limit
ax.tick_params(bottom=True, left=True) # Activating ticks of the x and y axes
ax.legend() # Legend to distinguish between the two groups

### Mehrere *Subplots* in einer *Figure*

`plt.subplots` kann auch mehrere *Axes* in einer *Figure* erzeugen und in einem Raster anlegen (Parameter: `nrows` bzw. `ncols`). Anstelle eines *Figure* und eines *Axes* Objektes gibt die Funktion jetzt eine *Figure* und ein Array aus *Axes* zurück:

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True, sharey=True)
print(type(axs))
print(len(axs))
axs[0].plot(data_mean.xs('Control'))
axs[0].set_title('Control')
axs[0].set_ylabel('Cortisol [nmol/l]')
axs[1].plot(data_mean.xs('Intervention'))
axs[1].set_title('Intervention')
axs[1].set_ylabel('Cortisol [nmol/l]')
axs[1].set_xlabel('Samples')
fig.tight_layout() # tight_layout() passt am Ende des Plottens noch Abstände zwischen Subplots, Achsen-Labels etc. an

### Verwenden der `plot`-Methode von DataFrames
Pandas Dataframes (und auch GroupBy-Objekte) besitzen eine `plot` methode, die direkt aufgerufen werden kann. Der Index wird dabei zur x-Achse, die Spalten zur y-Achse.

In [ ]:
data_mean.T

Transponieren des Dataframes, damit Samples zum Index (-> x-Achse) werden und Gruppen zu Spalten (-> y-Achse).
`plot()` gibt ein `Axes`-Objekt (oder ein Array aus `Axes`-Objekten) zurück, mit dem im Anschluss weitergearbeitet werden kann.

In [ ]:
ax = data_mean.T.plot()
# weitere Operationen auf Axes-Objekt
ax.set_xlabel("Sample")
ax.set_ylabel("Cortisol [nmol/l]")
ax.tick_params(bottom=True, left=True) # Ticks der x- und y-Achsen aktivieren

Der `plot()`-Methode kann auch ein bereits erzeuges `Axes`-Objekt übergeben werden, in welches dann hineingeplottet wird:

In [ ]:
fig, ax = plt.subplots()
data_mean.T.plot(ax=ax)
# weitere Operationen...

Konfiguration der `plot()`-Methode:

In [ ]:
ax = data_mean.T.plot(kind='bar') # Plot-Art

In [ ]:
axs = data_mean.T.plot(figsize=(8, 3), subplots=True, layout=(1,2), sharey=True) # Jede Spalte wird als eigener Subplot geplottet, 'layout' spezifiziert das Plot-Layout (Anzahl der Zeilen/Spalten)

### **Linksammlung**: Relevante Funktionen von Matplotlib

* Matplotlib Sample Plots: 
    * https://matplotlib.org/tutorials/introductory/sample_plots.html#sphx-glr-tutorials-introductory-sample-plots-py
    * https://matplotlib.org/gallery/index.html
* Artists: https://matplotlib.org/tutorials/intermediate/artists.html#sphx-glr-tutorials-intermediate-artists-py
    * Annotations: https://matplotlib.org/tutorials/text/annotations.html#sphx-glr-tutorials-text-annotations-py
    * Text: https://matplotlib.org/tutorials/text/text_intro.html#sphx-glr-tutorials-text-text-intro-py
* Legends: https://matplotlib.org/tutorials/intermediate/legend_guide.html#sphx-glr-tutorials-intermediate-legend-guide-py

In [ ]:
plt.close('all')

### **Beispiel**: Veränderung / Erweiterung von bestehenden Plots

Plots von Funktionen aus Bibliotheken (also Funktionen, die ein Axes-Objekt zurückliefern oder denen man ein Axes-Objekt übergibt, in welches dann geplottet wird), kann man – wie selbst erzeugte Plots – flexibel erweitern und verändern.

In [ ]:
import biopsykit.signals.ecg as ecg
from biopsykit.protocols import CFT

plt.rcParams['timezone'] = "Europe/Berlin" # Spezifikation der Zeitzone in den Matplotlib-Einstellungen, falls Plots mit Datetime-Achsen geplottet werden
cft = CFT()
# Laden der zu plottenden Daten
hr_dict = ecg.io.load_hr_subject_dict("data/hr_sample_mist.xlsx")
df_hr = hr_dict['MIST3']

# CFT-parameter berechnen (mehr dazu im nächsten Video):
params = cft.compute_cft_parameter(df_hr, return_dict=True)
params

In [ ]:
fig, ax = cft.cft_plot(data=df_hr, figsize=(10,5))

#### Manuelles Erstellen des Grund-Plots

CFT-Daten bestehen aus 3 Phasen:
* Baseline (0s - 60s)
* Cold Face Test-Intervention (60s - 180s)
* Recovery (180s - 240s)


Zuwerst wird der Index von absoluter Zeit (`datetime`) in relative Zeit (Beginn des Plots:`0 s`) umgewandelt.
Da der Datensatz länger als 240s ist (ab 240s beginnt der Stress-Test), wird der Rest abgeschnitten

In [ ]:
df_cft = df_hr.copy()
df_cft.index = (df_hr.index - df_hr.index[0]).total_seconds()

# Start and end times of CFT phases
plot_start = 0
cft_start = 60
cft_end = 180
plot_end = 240
df_cft = df_cft.loc[plot_start:plot_end]

df_cft.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(df_cft, color="#8D1429", lw=1.5)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Heart Rate [bpm]")
ax.tick_params(left=True, bottom=True)
ax.margins(x=0, y=0.1)
fig.tight_layout()

#### Implementierung einzelner Erweiterungen

Plot-Parameter:
* `ls` = linestyle
* `lw` = linewidth
* `alpha` = Transparenz
* `ha` = horizontal Alignment
* `va` = vertical Alignment
* `s` = Text

**Markieren der einzelnen Phasen mittels `axvspan` (Hintergrund) und `text` (Beschriftung)**

Mehr zu Plot-Koordinaten und Achsen-Transformationen: https://matplotlib.org/3.1.1/tutorials/advanced/transforms_tutorial.html

In [ ]:
ax.axvspan(xmin=plot_start, xmax=cft_start, color='#e0e0e0', alpha=0.5) # Baseline
ax.axvspan(xmin=cft_start, xmax=cft_end, color='#9e9e9e', alpha=0.5) # CFT
ax.axvspan(xmin=cft_end, xmax=plot_end, color='#757575', alpha=0.5) # Recovery

In [ ]:
ax.text(
    x=plot_start + 0.5 * (cft_start - plot_start), y=0.95, 
    transform=ax.get_xaxis_transform(), s="Baseline", ha='center', va='center', fontsize=14
)
ax.text(
    x=cft_start + 0.5 * (cft_end - cft_start), y=0.95, 
    transform=ax.get_xaxis_transform(), s="CFT", ha='center', va='center', fontsize=14
)
ax.text(
    x=cft_end + 0.5 * (plot_end - cft_end), y=0.95, 
    transform=ax.get_xaxis_transform(), s="Recovery", ha='center', va='center', fontsize=14
);

**Baseline-Herzrate**  
Horizontale Linie (`ax.hlines()` von `plot_start` bis `cft_end`)

In [ ]:
ax.hlines(y=params['baseline_hr'], xmin=plot_start, xmax=cft_end, ls='--', lw=2, alpha=0.6, color='#98a4ae');

**Peak Bradycardia** (niedrigste Herzrate)  
* Vertikale Linie an Zeitpunkt $x_{peak}$
* Marker an Stelle $(x_{peak}, y_{peak})$

In [ ]:
x_peak_idx = params['cft_start_idx'] + params['peak_brady_idx']
# Vertikale Linie
ax.axvline(x=df_cft.index[x_peak_idx], ls='--', lw=2, alpha=0.6, color='#003865');

In [ ]:
# Marker an einem Punkt
ax.plot(df_cft.index[x_peak_idx], df_cft.iloc[x_peak_idx], color='#003865', marker='o', markersize=7);

In [ ]:
# Viele, viele weitere Erweiterungsmöglichkeiten...

### Exportieren von Plots

Exportieren von Plots als Datei: `Figure.savefig(<filename>)`  
Wichtige Parameter:
* `format`: Dateiformat:
    * Pixelgrafik: png, jpeg (Angabe von `dpi` steuert die Auflösung)
    * Vektorgrafik (empfohlen, da Vektorgrafiken beim Heranzoomen nicht verpixeln und sich diese auch gut in Latex-Dokumenten einbinden lassen): `pdf`, `svg`. Empfohlen: `pdf`  
    => falls nicht spezifiziert wird Dateiformat aus dem Dateinamen erschlossen
* `dpi`: Auflösung der exportierten Grafik (bei Pixelgrafiken)
* `transparent`: `True`, falls weißer Figure-Hintergrund transparent exportiert werden soll, `False`, falls Hintergrund weiß sein soll
* ...

In [ ]:
fig.savefig("img/cft_test.pdf", format='pdf', transparent=True)
fig.savefig("img/cft_test.png", format='png', transparent=True, dpi=300)

## Seaborn

Seaborn (https://seaborn.pydata.org/): Python-Visualisierungsbibliothek, die auf Matplotlib aufbaut. Seaborn bietet viele *high-level* Plot-Funktionen, die die *low-level* Funktionen von Matplotlib wrappen und ist vor allem für die Visualisierung von statistischen Daten geeignet.

Import: `import seaborn as sns`

Seaborn bietet verschiedenste Plot-Funktionen für verschiedene Datentypen:
* Beziehungen von Daten: `sns.relplot()` (`scatterplot`, `lineplot`, ...)
* Verteilungen von Daten: `sns.distplot()` (`histplot`, `kdeplot`, `rugplot`, ...)
* Kategorien von Daten: `sns.catplot()` (`barplot`, `swarmplot`, `boxplot`, `violinplot`, ...)

### Seaborn Styles und Context

Seaborn bietet viele Funktionen, um das Aussehen eines Plots schnell und einfach zu verändern (https://seaborn.pydata.org/tutorial/aesthetics.html):
* Context: Kontrolliert die Linienstärke, Textgröße, etc.  
    Vordefinierte Contexts: `notebook` (default), `paper`, `talk`, `poster`  
    => Ideal, um schnell verschieden skalierte Plots für Paper und Präsentation zu erzeugen
* Style:  
    Vordefinierte Styles: `darkgrid`, `whitegrid`, `dark`, `white`, und `ticks`  
    => Abstrahiert verschiedene Style-Ästhetiken (Ticks, Schriftart, Grid, etc.), die in Matplotlib sonst "manuell" zu jedem Plot hinzugefügt werden müssen (oder in der Matplotlib-Konfiguration definiert werden müssen)
* Color Palette:
    Vordefinierte Farbpaletten und Funktionen, um Farbpaletten zu erzeugen: https://seaborn.pydata.org/tutorial/color_palettes.html

In [ ]:
sns.set_theme(context='notebook', style='ticks')
# Eine der vordefinierten Farbpaletten
# palette = sns.color_palette("Paired")
# Alternative: Erstellen einer Farb-Palette mit dem FAU-Blau als Grundfarbe und weiteren, helleren Farbabstufungen.
# Achtung: Diese Palette ist von "hell" nach "dunkel angeordnet" => Umkehren der Reihenfolge, damit FAU-blau erste Farbe ist
palette = sns.color_palette("light:#003865", n_colors=4)[::-1]

# Setzen der Palette
sns.set_palette(palette)
# Anzeigen der Palettenfarben
sns.color_palette(palette)

<div class="alert alert-block alert-info">
    <b>Hinweis:</b> Das Setzen von Context-, Style- oder anderer Parameter mittels Seaborn oder Matplotlib wirkt sich auf <b>alle</b> Plots im Notebook aus, auch auf bereits erstelle, sobald diese aktualisiert werden!
</div>

Beispiel von Anfang:

In [ ]:
fig, ax = plt.subplots() # Erzeugen eines Figure-Objekts und einer zugehörigen Axes
ax.plot(x, y)
ax.set_xlabel("x-Achse")
ax.set_ylabel("y-Achse")
ax.set_title("Einfacher Plot – Seaborn-Style")
fig.tight_layout()

### Datenformat

(Fast) alle Seaborn-Funktionen funktionieren am besten mit pandas Dataframes im *Long-Format*. Dadurch können einzelne Index-Spalten spezifiziert werden, um verschiedene Plots zwischen Gruppen, Zeitpunkten, Variablen, etc. zu erzeugen

Laden der Beispiel-Cortisol-Daten und Berechnen von Standard-Features

In [ ]:
data = bp.example_data.get_saliva_example()
data_features = bp.saliva.standard_features(data)
data_features.head()

Umwandeln der beiden DataFrames ins *Long-Format*

In [ ]:
data_long = pd.wide_to_long(
    data_features.reset_index(), 
    stubnames="cortisol", 
    sep='_', 
    i=['subject', 'condition'], 
    j='feature', 
    suffix=r"\w+"
)
display(data.head())
display(data_long.head())

### Beziehungen von Daten

#### Lineplot

https://seaborn.pydata.org/generated/seaborn.lineplot.html#seaborn.lineplot

Plotten des mittleren Cortisol-Verlaufs ± 95% CI als Error-Band

In [ ]:
data

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.lineplot(data=data.reset_index(), x='sample', y='cortisol', ax=ax)
fig.tight_layout()

Standardabweichung statt 95%-CI als Errorband: `ci='sd'`, Deaktivieren des Errorbands: `ci=None`

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.lineplot(data=data.reset_index(), x='sample', y='cortisol', ax=ax, ci='sd')
fig.tight_layout()

Beide Gruppen separat geplottet: `hue` Parameter (verschiedene Farben) (alternativ: `style` Parameter für verschiedene Styles oder `hue` *und* `style` für Kombination)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.lineplot(data=data.reset_index(), x='sample', y='cortisol', hue='condition', ax=ax)
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.lineplot(data=data.reset_index(), x='sample', y='cortisol', hue='condition', style='condition', ax=ax)
ax.set_ylabel(r"Cortisol $\left[\frac{nmol}{l}\right]$")
fig.tight_layout()

Beide Gruppen in separaten Subplots: Figure-Level Funktion `sns.relplot(kind='line')` mit Parameter `col`

Unterschied Figure-Level vs. Axes-Level Funktionen: https://seaborn.pydata.org/tutorial/function_overview.html#figure-level-vs-axes-level-functions

<div class="alert alert-block alert-info">
    <b>Hinweis:</b> <code>sns.relplot()</code> ist eine <i>Figure-Level</i> Funktion und liefert daher ein Objekt vom Typ <code>seaborn.FacetGrid</code> zurück.
</div>

In [ ]:
g = sns.relplot(data=data.reset_index(), x='sample', y='cortisol', col='condition', kind='line')
g.fig.tight_layout()

#### Scatterplot

https://seaborn.pydata.org/generated/seaborn.scatterplot.html#seaborn.scatterplot

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.scatterplot(data=data_features.reset_index(), x='cortisol_mean', y='cortisol_std', hue='condition', ax=ax)
fig.tight_layout()

### Verteilungen von Daten

Verteilung des Ortes des Cortisol-Maximums, aufgeteilt auf die beiden Gruppen

#### Histogramm

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.histplot(data=data_features.reset_index(), x='cortisol_argmax', hue='condition', ax=ax)
fig.tight_layout()

Verteilung der Herzrate in einem Zeitintervall

In [ ]:
df_hr = pd.read_excel("data/hr_sample_mist.xlsx", sheet_name=1, index_col='time')
df_hr.head()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.histplot(data=df_hr, x='ECG_Rate', ax=ax)
fig.tight_layout()

Feste Größe an Behältern (z.B. 10)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.histplot(data=df_hr, x='ECG_Rate', bins=10, ax=ax)
fig.tight_layout()

Kumulative Häufigkeit

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.histplot(data=df_hr, x='ECG_Rate', cumulative=True, ax=ax)
fig.tight_layout()

y-Achse: Wahrscheinlichkeit des Behälters (statt absoluter Anzahl)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.histplot(data=df_hr, x='ECG_Rate', stat='probability', ax=ax)
fig.tight_layout()

#### Kernel Density Estimation (KDE)

Histogramm + KDE-Plot (KDE = Schätzung der den Daten zugrundeliegenden Wahrscheinlichkeitsdichteverteilung)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.histplot(data=df_hr, x='ECG_Rate', stat='probability', alpha=0.5, ax=ax)
sns.kdeplot(data=df_hr, x='ECG_Rate', ax=ax)
fig.tight_layout()

Generieren einer willkürlichen Gruppe (Gruppe 0 = Daten von Minute 0 bis Minute 3, Gruppe 1 = Daten von Minute 3 bis Ende) zur Illustration:

In [ ]:
df_hr['group'] = df_hr.index.isin(df_hr.first("3min").index).astype(int)

Plotten zweier Histogramme + KDE

In [ ]:
df_hr

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.histplot(data=df_hr, x='ECG_Rate', stat='probability', hue='group', alpha=0.5, ax=ax)
sns.kdeplot(data=df_hr, x='ECG_Rate', hue='group', ax=ax)
fig.tight_layout()

### Kategorien von Daten

#### Barplot

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.barplot(data=data_features.reset_index(), x='condition', y='cortisol_mean', ax=ax, order=['Control', 'Intervention'])
fig.tight_layout()

#### Boxplot

Bedeutung der einzelnen Boxplot-Komponenten:

In [ ]:
from IPython.display import Image
Image("img/img_boxplot_explanation.jpg")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.boxplot(data=data_features.reset_index(), x='condition', y='cortisol_mean', ax=ax)
fig.tight_layout()

Boxplots können auch mit Notch (Parameter `notch=True`) geplottet werden. Der Notch kennzeichnet das **95%-CI des Medians** und kann daher als schnelle Orientierungshilfe herangezogen werden, ob ein signifikanter Unterschied vorliegen **könnte**:
* Überlappen sich die Notches der beiden Gruppen **nicht**, so **kann** der Gruppenunterschied signifikant sein: "If two boxes' notches do not overlap there is ‘strong evidence’ (95% confidence) their medians differ"
* Überlappen sie sich, ist der Unterschied auf alle Fälle **nicht** signifikant.

=> Boxplot mit Notches sehen allerdings nicht so schön aus (v.a., wenn das 95%-CI größer ist als die IQR) und sollten daher nur für eine kurze visuelle Inspektion der Ergebnisse aktiviert werden und nicht in den *finalen* Plots.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.boxplot(data=data_features.reset_index(), x='condition', y='cortisol_argmax', notch=True, ax=ax)
fig.tight_layout()

Daten im Long-Format: *x-Achse* ist das berechnete Feature, *y-Achse* der Wert des jeweiligen Features, *hue* die Gruppenzugehörigkeit

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.boxplot(data=data_long.reset_index(), x='feature', y='cortisol', hue='condition', notch=True, ax=ax)
fig.tight_layout()

Festlegen der Reihenfolge der Bedingungen `hue_order`

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.boxplot(data=data_long.reset_index(), x='feature', y='cortisol', hue='condition', hue_order=['Control', 'Intervention'], notch=True, ax=ax)
fig.tight_layout()

Boxplots ausgewählter Features in eigenen Subplots

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(10, 5))
sns.boxplot(data=data_features.reset_index(), x='condition', y='cortisol_argmax', ax=axs[0])
sns.boxplot(data=data_features.reset_index(), x='condition', y='cortisol_mean', ax=axs[1])
sns.boxplot(data=data_features.reset_index(), x='condition', y='cortisol_std', ax=axs[2])

axs[0].set_ylabel(r"$argmax_{Cortisol}~[AU]$")
axs[1].set_ylabel(r"$Cortisol~[nmol/l]$")
fig.tight_layout()

### Violin-Plot

Mischung aus Boxplot und Density-Plot

In [ ]:
df_hr

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.violinplot(data=df_hr, x='group', y='ECG_Rate', ax=ax)
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.violinplot(data=data_long.reset_index(), x='feature', y='cortisol', hue='condition', ax=ax)
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.violinplot(data=data_long.reset_index(), x='feature', y='cortisol', hue='condition', split=True, inner='quartile', ax=ax)
fig.tight_layout()